In [ ]:
!pip3 install pytelegrambotapi

In [ ]:
import telebot
from PIL import Image

# Подключаем библиотеку PIL Python Imaging Library Она нужна для работы с изображениями Мы будем использовать её чтобы открывать и изменять размеры изображений

import requests
from io import BytesIO

# Подключаем модуль BytesIO из библиотеки io Он позволяет работать с данными в виде байтов Мы будем использовать его чтобы сохранять изображения в памяти не на жестком диске

import numpy as np
import tensorflow as tf

# Подключаем библиотеку tensorflow Это мощная библиотека для машинного обучения Мы будем использовать её чтобы загрузить предобученную модель для распознавания изображений

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions

# Из TensorFlow мы подключаем три вещи
# MobileNetV2 это предобученная модель которая умеет распознавать объекты на изображениях
# preprocess_input функция которая готовит изображение для модели например меняет цвета и размеры
# decode_predictions функция которая переводит результаты модели в понятные категории например пластиковая бутылка

# Создаем экземпляр бота
bot = telebot.TeleBot('7944203173:AAF0TyZE2DmAkSLq8RTBdV8UFtKIzcCW9rE')

# Загружаем предобученную модель MobileNetV2
model = MobileNetV2(weights='imagenet')

# Функция для классификации изображения
def classify_image(image):
    # Преобразуем изображение в массив NumPy
    image = image.resize((224, 224))  # Размер входного изображения для MobileNetV2
    image_array = np.array(image)
    image_array = np.expand_dims(image_array, axis=0)  # Добавляем размерность batch
    image_array = preprocess_input(image_array)  # Нормализуем пиксели

    # Получаем предсказания модели
    predictions = model.predict(image_array)
    decoded_predictions = decode_predictions(predictions, top=1)[0]  # Топ-1 категория

    # Возвращаем название категории и вероятность
    category_name = decoded_predictions[0][1]
    confidence = decoded_predictions[0][2]
    return category_name, confidence

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Программа по сортировке отходов запущена. Пришлите изображение, на котором есть отходы, а бот определит категорию мусора.')

# Получение текстовых сообщений от пользователя
@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, 'Вы написали: ' + message.text)
    print(f"Получено текстовое сообщение: {message.text}")

# Получение изображений от пользователя
@bot.message_handler(content_types=["photo"])
def handle_photo(message):
    try:
        # Получаем ID файла изображения
        file_id = message.photo[-1].file_id
        bot.send_message(message.chat.id, 'Изображение получено.')

        # Получаем информацию о файле
        file_info = bot.get_file(file_id)
        file_url = f'https://api.telegram.org/file/bot{bot.token}/{file_info.file_path}'

        # Загружаем изображение с помощью requests
        response = requests.get(file_url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))

            # Классифицируем изображение
            category_name, confidence = classify_image(image)

            # Отправляем результат пользователю
            result_message = f'Категория мусора: {category_name} (вероятность: {confidence:.2f})'
            bot.send_message(message.chat.id, result_message)

            # Отправляем изображение обратно пользователю
            # bot.send_photo(message.chat.id, photo=BytesIO(response.content))
        else:
            bot.send_message(message.chat.id, 'Не удалось загрузить изображение.')

    except Exception as e:
        print(f"Произошла ошибка: {e}")
        bot.send_message(message.chat.id, 'Произошла ошибка при обработке изображения.')

# Запускаем бота
try:
    bot.polling(none_stop=True, interval=0)
except KeyboardInterrupt:
    print("Бот остановлен.")